### Imports

In [4]:
import re
import requests
import pandas as pd
import json
import pdfplumber
from PyPDF2 import PdfReader
import requests
from io import BytesIO

In [5]:
import os
os.getcwd()

'c:\\Users\\ilian\\Documents\\Projects\\git_projects\\university\\WieWaehltMeinAbgeordneter-main\\WieWaehltMeinAbgeordneter\\extractor'

# Extract

### Paths

In [6]:
# Dir paths
cwd = os.path.join(os.getcwd())
samples_dir = os.path.join(cwd, 'samples')
results_dir = os.path.join(cwd, 'results')

# Pair csv
pdf_xls_abstimmung = os.path.join(cwd, 'pdf_xls_paths.csv')
updated_pdf_xls_abstimmung = os.path.join(cwd, 'updated_abstimmungen.csv')

# Data load paths
path_xlsx_abstimmung = os.path.join(samples_dir, 'sample_abstimmungsliste.xlsx')
path_pdf_abstimmung = os.path.join(samples_dir, 'sample_abstimmungsliste.pdf')
path_pdf_drucksache = os.path.join(samples_dir, 'sample_drucksache.pdf')

# Save paths
path_csv_abstimmung = os.path.join(results_dir, 'abstimmung_ergebnisse.csv')
path_json_abstimmung = os.path.join(results_dir, 'abstimmung_ergebnisse.json')
path_json_drucksache = os.path.join(results_dir, 'sample_drucksache.json')

### Abstimmungsliste

##### XLSX

In [4]:
# Load xlsx
xlsx_abstimmungsliste = pd.read_excel(path_xlsx_abstimmung)

# Filter results
df_filtered = xlsx_abstimmungsliste[["Fraktion/Gruppe", "Name", "Vorname", "ja", "nein", "Enthaltung", "ungültig", "nichtabgegeben"]]
df_filtered["Ergebnis"] = df_filtered.apply(
    lambda row: f"Ja: {row['ja']}, Nein: {row['nein']}, Enthaltungen: {row['Enthaltung']}, Nicht abgegeben: {row['nichtabgegeben']}", axis=1
)

# Display
display(df_filtered)

# Save
df_filtered.to_csv(path_csv_abstimmung, index=False)
df_filtered.to_json(path_json_abstimmung, orient="records", lines=True)

C:\Users\ilian\AppData\Local\Temp\ipykernel_22280\2324430944.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["Ergebnis"] = df_filtered.apply(


,Fraktion/Gruppe,Name,Vorname,ja,nein,Enthaltung,ungültig,nichtabgegeben,Ergebnis
0,SPD,Abdi,Sanae,1,0,0,0,0,"Ja: 1, Nein: 0, Enthaltungen: 0, Nicht abgegeb..."
1,SPD,Ahmetovic,Adis,0,0,0,0,1,"Ja: 0, Nein: 0, Enthaltungen: 0, Nicht abgegeb..."
2,SPD,Alabali-Radovan,Reem,1,0,0,0,0,"Ja: 1, Nein: 0, Enthaltungen: 0, Nicht abgegeb..."
3,SPD,Andres,Dagmar,1,0,0,0,0,"Ja: 1, Nein: 0, Enthaltungen: 0, Nicht abgegeb..."
4,SPD,Annen,Niels,1,0,0,0,0,"Ja: 1, Nein: 0, Enthaltungen: 0, Nicht abgegeb..."
...,...,...,...,...,...,...,...,...,...
728,Fraktionslos,Seidler,Stefan,1,0,0,0,0,"Ja: 1, Nein: 0, Enthaltungen: 0, Nicht abgegeb..."
729,Fraktionslos,Seitz,Thomas,0,1,0,0,0,"Ja: 0, Nein: 1, Enthaltungen: 0, Nicht abgegeb..."
730,Fraktionslos,Spaniel,Dirk,0,0,0,0,1,"Ja: 0, Nein: 0, Enthaltungen: 0, Nicht abgegeb..."
731,Fraktionslos,Wissing,Volker,1,0,0,0,0,"Ja: 1, Nein: 0, Enthaltungen: 0, Nicht abgegeb..."


##### PDF

In [5]:
# Load pdf
with pdfplumber.open(path_pdf_abstimmung) as pdf:
    text = ""
    for page in pdf.pages:
        text += page.extract_text()

# Extract results
ja_stimmen = re.search(r"Ja-Stimmen: (\d+)", text).group(1)
nein_stimmen = re.search(r"Nein-Stimmen: (\d+)", text).group(1)
enthaltungen = re.search(r"Enthaltungen: (\d+)", text).group(1)
nicht_abgegeben = re.search(r"Nicht abgegebene Stimmen: (\d+)", text).group(1)

gesetzesentwurf = re.search(r"Gesetzentwurf der (.+)", text).group(1)
drucken_sachen = re.findall(r"Drucksachen (\d+/\d+)", text)

# Display results
print(f"Ja-Stimmen: {ja_stimmen}")
print(f"Nein-Stimmen: {nein_stimmen}")
print(f"Enthaltungen: {enthaltungen}")
print(f"Nicht abgegebene Stimmen: {nicht_abgegeben}")
print(f"Gesetzentwurf: {gesetzesentwurf}")
print(f"Drucksachen: {drucken_sachen}")

Ja-Stimmen: 600
Nein-Stimmen: 69
Enthaltungen: 0
Nicht abgegebene Stimmen: 64
Gesetzentwurf: Fraktionen SPD, CDU/CSU, BÜNDNIS 90/DIE GRÜNEN und FDP sowie
Drucksachen: ['20/12977']


In [6]:
# JSON save structure
data = {
    "gesetz": {
        "gesetzentwurf": "Fraktionen SPD, CDU/CSU, BÜNDNIS 90/DIE GRÜNEN und FDP",
        "drucksachen": ["20/12977", "20/14302"]
    },
    "abstimmungsstatistik": {
        "ja_stimmen": ja_stimmen,
        "nein_stimmen": nein_stimmen,
        "enthaltungen": enthaltungen,
        "nicht_abgegeben": nicht_abgegeben
    }
}

In [7]:
with open(path_json_drucksache, "w") as json_file:
    json.dump(data, json_file, indent=4)

### Drucksache (associated files)

In [8]:
# Load pdf
with pdfplumber.open(path_pdf_drucksache) as pdf:
    text = ""
    for page in pdf.pages:
        text += page.extract_text()


In [9]:
# Extract sections
sections = {
    "Problem und Ziel": r"A\. Problem und Ziel\s+(.*?)\s+B\.",
    "Lösung": r"B\. Lösung\s+(.*?)\s+C\.",
    "Alternativen": r"C\. Alternativen\s+(.*?)\s+D\.",
    "Haushaltsausgaben ohne Erfüllungsaufwand": r"D\. Haushaltsausgaben ohne Erfüllungsaufwand\s+(.*?)\s+E\.",
    "Erfüllungsaufwand": r"E\. Erfüllungsaufwand\s+(.*?)\s+F\."
}

extracted_sections = {}
for section, pattern in sections.items():
    match = re.search(pattern, text, re.DOTALL)
    if match:
        extracted_sections[section] = match.group(1).strip()


In [10]:
# Save to JSON
with open(path_json_drucksache, "w", encoding="utf-8") as json_file:
    json.dump(extracted_sections, json_file, indent=4, ensure_ascii=False)

# Print results
for section, content in extracted_sections.items():
    print(f"### {section} ###\n{content}\n")

### Problem und Ziel ###
Das Grundgesetz hat sich für eine starke Verfassungsgerichtsbarkeit entschieden.
Diese zentrale Weichenstellung lässt sich an den – auch im internationalen Ver-
gleich – weitreichenden Entscheidungsbefugnissen ablesen, die das Grundgesetz
dem Bundesverfassungsgericht zuweist. Auf dieser Grundlage hat sich das Bun-
desverfassungsgericht als Garant der freiheitlich-demokratischen Ordnung und
als für Staat und Gesellschaft wesentliches Verfassungsorgan mittlerweile fest
etabliert.
Als das Grundgesetz am 24. Mai 1949 in Kraft trat, war die neuartige Institution
„Bundesverfassungsgericht“ verfassungsrechtlich nur teilweise näher ausgeformt.
Auch das Verständnis des Gerichts als Verfassungsorgan war anfangs noch nicht
allgemein konsentiert. Es oblag in der Folge sehr weitgehend dem einfachen Ge-
setzgeber, Stellung und Struktur des Gerichts näher zu regeln. Dieser Aufgabe ist
der Gesetzgeber mit dem Gesetz über das Bundesverfassungsgericht (Bundesver-
fassungsgericht

### Get pdf infos

In [7]:
def read_csv(csv_path):
    return pd.read_csv(csv_path)


def download_pdf(pdf_url):
    response = requests.get(pdf_url)
    response.raise_for_status()
    return BytesIO(response.content)


def extract_pdf_info(pdf_content):
    reader = PdfReader(pdf_content)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    
    is_gesetzesentwurf = 'Gesetzentwurf der Fraktionen' in text
    dokumentnummer = re.findall(r'(1[0-9]|20)/\d+', text)
    datum_match = re.search(r', den (\d{1,2}\.\d{1,2}\.\d{4})', text)
    datum = datum_match.group(1) if datum_match else None

    return is_gesetzesentwurf, dokumentnummer, datum


def update_csv_with_pdf_info(csv_path, sample_size=None):
    df = read_csv(csv_path)
    df['Gesetzentwurf'] = None
    df['Dokumentennummer'] = None
    df['Datum'] = None
    
    rows_to_process = df.head(sample_size) if sample_size else df
    
    for idx, row in rows_to_process.iterrows():
        pdf_url = row['pdf']
        try:
            pdf_content = download_pdf(pdf_url)
            is_gesetzesentwurf, dokumentnummer, datum = extract_pdf_info(pdf_content)
            df.at[idx, 'Gesetzentwurf'] = is_gesetzesentwurf
            df.at[idx, 'Dokumentennummer'] = dokumentnummer
            df.at[idx, 'Datum'] = datum
        except Exception as e:
            print(f"Fehler beim Verarbeiten der PDF für Zeile {idx}: {e}")
    
    updated_csv_path = "updated_abstimmungen.csv"
    df.to_csv(updated_csv_path, index=False)
    return df


In [8]:
sample_size = None

In [9]:
updated_df = update_csv_with_pdf_info(pdf_xls_abstimmung, sample_size)
updated_df.head(sample_size)


,Unnamed: 0,xls,pdf,Gesetzentwurf,Dokumentennummer,Datum
0,0,https://www.bundestag.de/resource/blob/1025330...,https://www.bundestag.de/resource/blob/1025328...,False,"[20/11626, 20/13413]",18.10.2024
1,1,https://www.bundestag.de/resource/blob/1025326...,https://www.bundestag.de/resource/blob/1025324...,False,"[20/12976, 20/13413]",18.10.2024
2,2,https://www.bundestag.de/resource/blob/1025318...,https://www.bundestag.de/resource/blob/1025316...,False,"[20/12805, 20/13413, 20/13417]",18.10.2024
3,3,https://www.bundestag.de/resource/blob/1025312...,https://www.bundestag.de/resource/blob/1025310...,False,"[20/12805, 20/13413, 20/13416]",18.10.2024
4,4,https://www.bundestag.de/resource/blob/1025308...,https://www.bundestag.de/resource/blob/1025306...,False,"[20/12805, 20/13413, 20/13415]",18.10.2024
...,...,...,...,...,...,...
689,689,https://www.bundestag.de/resource/blob/192686/...,https://www.bundestag.de/resource/blob/192682/...,False,[],None
690,690,https://www.bundestag.de/resource/blob/192678/...,https://www.bundestag.de/resource/blob/192688/...,False,[],None
691,691,https://www.bundestag.de/resource/blob/192676/...,https://www.bundestag.de/resource/blob/192680/...,False,[],None
692,692,https://www.bundestag.de/resource/blob/192674/...,https://www.bundestag.de/resource/blob/192672/...,False,[],None


### Get associated Drucksachen

In [10]:
api_url = 'https://search.dip.bundestag.de/api/v1/drucksache-text'
apikey = 'I9FKdCn.hbfefNWCY336dL6x62vfwNKpoN2RZ1gp21'

In [11]:
def read_csv(csv_path):
    return pd.read_csv(csv_path)

def get_associated_drucksachen(dokumentnummer_list, api_url='https://search.dip.bundestag.de/api/v1/drucksache-text', apikey='I9FKdCn.hbfefNWCY336dL6x62vfwNKpoN2RZ1gp21'):
    params = {
        'f.dokumentnummer': dokumentnummer_list,
        'format': 'json',
        'apikey': apikey
    }
    response = requests.get(api_url, params=params)
    
    if response.status_code == 200:
        api_data = response.json()
        return api_data
    else:
        return None

def process_csv_and_fetch_data(csv_path, sample_size=None):
    df = read_csv(csv_path)
    df['Associated_PDF_URLs'] = None
    
    rows_to_process = df.head(sample_size) if sample_size else df
    
    for idx, row in rows_to_process.iterrows():
        dokumentnummer_list = row['Dokumentennummer']
        
        if isinstance(dokumentnummer_list, str):
            dokumentnummer_list = dokumentnummer_list.strip("[]").replace("'", "").split(", ")
        
        if dokumentnummer_list:
            associated_pdf_urls = []
            api_data = get_associated_drucksachen(dokumentnummer_list)
            
            if api_data and 'documents' in api_data:
                for doc in api_data['documents']:
                    if doc['drucksachetyp'] == 'Gesetzentwurf' and 'fundstelle' in doc:
                        associated_pdf_urls.append(doc['fundstelle']['pdf_url'])
                
                if associated_pdf_urls:
                    df.at[idx, 'Associated_PDF_URLs'] = json.dumps(associated_pdf_urls)

    df.to_csv(updated_pdf_xls_abstimmung, index=False)
    return df


In [12]:
updated_df = process_csv_and_fetch_data(updated_pdf_xls_abstimmung, sample_size)
updated_df.head(sample_size)

,Unnamed: 0,xls,pdf,Gesetzentwurf,Dokumentennummer,Datum,Associated_PDF_URLs
0,0,https://www.bundestag.de/resource/blob/1025330...,https://www.bundestag.de/resource/blob/1025328...,False,"['20/11626', '20/13413']",18.10.2024,None
1,1,https://www.bundestag.de/resource/blob/1025326...,https://www.bundestag.de/resource/blob/1025324...,False,"['20/12976', '20/13413']",18.10.2024,None
2,2,https://www.bundestag.de/resource/blob/1025318...,https://www.bundestag.de/resource/blob/1025316...,False,"['20/12805', '20/13413', '20/13417']",18.10.2024,"[""https://dserver.bundestag.de/btd/20/128/2012..."
3,3,https://www.bundestag.de/resource/blob/1025312...,https://www.bundestag.de/resource/blob/1025310...,False,"['20/12805', '20/13413', '20/13416']",18.10.2024,"[""https://dserver.bundestag.de/btd/20/128/2012..."
4,4,https://www.bundestag.de/resource/blob/1025308...,https://www.bundestag.de/resource/blob/1025306...,False,"['20/12805', '20/13413', '20/13415']",18.10.2024,"[""https://dserver.bundestag.de/btd/20/128/2012..."
...,...,...,...,...,...,...,...
689,689,https://www.bundestag.de/resource/blob/192686/...,https://www.bundestag.de/resource/blob/192682/...,False,[],NaN,None
690,690,https://www.bundestag.de/resource/blob/192678/...,https://www.bundestag.de/resource/blob/192688/...,False,[],NaN,None
691,691,https://www.bundestag.de/resource/blob/192676/...,https://www.bundestag.de/resource/blob/192680/...,False,[],NaN,None
692,692,https://www.bundestag.de/resource/blob/192674/...,https://www.bundestag.de/resource/blob/192672/...,False,[],NaN,None
